#  Aula sobre processamento de dados matriciais


Neste Exemplo são apresentados métodos de leitura de arquivos matriciais de vários arquivos em  uma pasta para então processar todos os arquivo. Um ponto importante é que os dados foram lidos como array multidimensional, portanto sem informação espacial, porém o arquivo de saída foi  transformado em um Geotiff.


In [ ]:
import numpy as np

from osgeo import gdal
from osgeo import osr
from gdalconst import *

import os
import shutil

import glob
import matplotlib.pyplot as plt
%matplotlib inline

 Exemplo de um caminho completo para chegar a um arquivo no windows.
 
 Note que as barras estão todas apresentadas conforme o windows explorer mostra 
 em um copiar e colar. 
 
 O r em frente a string serve para forçar a leitura RAW da string.
 
 
 Isto é uma forma do python tirar a força dos caracteres especiais que podem estar contidos na string.
 
 
 Por exemplo: 
     \t é a representacao especial de uma tabulacao, 
     \n de nova linha.


#### arquivo_entrada = r"c:\Documents\dados\foc_sev_20161010.bin"

 Outras formas de endereçar um arquivo no windows sem dar problemas no problemas são:
 
     1) adicionar duas barras "c:\\Documents\\dados\\foc_sev_20161010.bin"
 
     2) trocar por barra invertida "c:/Documents/dados/foc_sev_20161010.bin"

 Em Linux e MacOS o caminho padrão é sempre a barra invertida

#### arquivo_entrada = "dados_exemplo/focos/2010_09_grd_focos.bin"

In [ ]:
# Verificar diretorio de trabalho
os.getcwd()

In [ ]:
# Modificar o diretorio de trabalho para o local onde estão os dados de interesse
os.chdir('/Users/fabianomorelli/ownCloud/notebooks/ipython_notebooks/dados_exemplo/')

In [ ]:
# Confirmar a mudança do diretorio de trabalho
os.getcwd()

In [ ]:
# Listar conteudo
os.listdir()

In [ ]:
focos = np.fromfile("./foc_sev_20161013.bin", dtype=np.int16)
# ou 
# focos = np.fromfile(caminho/completo/do/arquivo_entrada, dtype=np.int16)

In [ ]:
# Verificar se os dados foram lidos corretamente mostrando o maior valor do arquivo
focos.max()

In [ ]:
# Apresentar o menor valor do arquivo
focos.min()

In [ ]:
# Modificar o array para uma matriz com 6300 linhas e 5000 colunas
focos = focos.reshape(6300,5000)

# Outra forma de fazer a mudança é utilizando o metodos resize
# focos.resize((6300,5000))

<hr>

# Exercício - Fazer uma imagem com o somatório da quantidade de focos por píxel.

### 1 -  Abrir todos os arquivos de focos de um diretório 
### 2 - Sequencialmente fazer a soma dos arquivos
### 3 - Salvar o resultado em um arquivo em formato ENVI

In [ ]:
# Confirmacao do local de trabalho
os.getcwd()

In [ ]:
# Criar um novo arquivo com as mesmas características daqueles de interesse
novo = np.zeros_like(focos)

# Uma alternativa 
# novo = np.zeros((6300,5000))

In [ ]:
# Utilizar a biblioteca GLOB para trabalhar apenas com os arquivos específicos
glob.glob("./f*bin")

In [ ]:
# Processamento dos dados
for nomedoarquivo in  glob.glob("./f*bin"):
    print(nomedoarquivo)
    focos = np.fromfile(nomedoarquivo, dtype=np.int16).reshape(6300,5000)
    novo += focos  # este é o mesmo que fazer ==>     novo = novo + focos

print("Final do Processamento!!!")

In [ ]:
novo.max()

<hr>

# Transformação de um array para um dado georreferênciado


In [ ]:
# Definição dos parâmetros de transformação geométrica
geotransform = (-83.00, 0.01, 0.0, 13.00, 0.0, -0.01)

# Definição do Sistema de Referência Espacial
projecao = osr.SRS_WKT_WGS84

In [ ]:
# Verificação da Variável: geotransform
print(geotransform)

In [ ]:
# Verificação da Variável: projecao
print(projecao)

In [ ]:
# definições de parâmetros para criar o arquivo de saída
driver = gdal.GetDriverByName("GTIFF") # pode ser GTiff, ENVI, etc...
xsize = 5000
ysize = 6300
nbandas = 1
arquivo_saida = "/tmp/teste.bin"

In [ ]:
# Utilizar os comando de manipulacao de diretorio e arquivos para verificar se já existe
os.path.exists(arquivo_saida)

In [ ]:
# Caso o arquivo exista apagar antes de criar um novo
if os.path.exists(arquivo_saida):
    os.remove(arquivo_saida)
    os.remove(arquivo_saida[:-4] + ".hdr")
    print("Arquivo removido com sucesso!")
else:    ## Show an error ##
    print("Erro: %s arquivo nao encontrado" % arquivo_saida)

In [ ]:
# definicoes do arquivo de saida
dst_ds = driver.Create(arquivo_saida, xsize,ysize, nbandas,gdal.GDT_Int16)

In [ ]:
dst_ds.SetGeoTransform(geotransform)
dst_ds.SetProjection(projecao)

In [ ]:
dst_ds.GetRasterBand(1).WriteArray(novo) # note que aqui estamos usando o array que será gravado
dst_ds = None

<hr>

# Verificação conforme a aula de gdal

In [ ]:
dataset = gdal.Open("/tmp/teste.bin", GA_ReadOnly )

In [ ]:
print ('Driver: ', dataset.GetDriver().ShortName,'/', \
      dataset.GetDriver().LongName)
print ('Size is ',dataset.RasterXSize,'x',dataset.RasterYSize, \
      'x',dataset.RasterCount)
print ('Projection is ',dataset.GetProjection())

geotransform = dataset.GetGeoTransform()
if not geotransform is None:
    print ('Origin = (',geotransform[0], ',',geotransform[3],')')
    print ('Pixel Size = (',geotransform[1], ',',geotransform[5],')')

In [ ]:
band = dataset.GetRasterBand(1)

print ('Band Type=',gdal.GetDataTypeName(band.DataType))

min = band.GetMinimum()
max = band.GetMaximum()

if min is None or max is None:
    print("calcula estatistica")
    (min,max) = band.ComputeRasterMinMax(1)
    
print ('Min=%.3f, Max=%.3f' % (min,max))


<hr>

# Final da aula 20